# High Value Customers Identification (Insiders) #
**By: Marx Cerqueira**

# IMPORTS

In [1]:
import re
import os
import inflection
import sqlite3

import numpy   as np
import pandas  as pd
import seaborn as sns

import umap.umap_ as umap

from matplotlib import pyplot as plt

from sklearn import metrics       as m
from sklearn import preprocessing as pp
from sklearn import decomposition as dd
from sklearn import ensemble      as en
from sklearn import manifold      as mn
from sklearn import mixture       as mx
from sklearn import cluster       as c

from scipy.cluster import hierarchy as hc

from plotly import express as px
from sqlalchemy import create_engine

## Loading Data

In [2]:
# load data
df_ecomm_raw = pd.read_csv('/home/marxcerqueira/repos/Kaggle-HighValue-Custormers-Identification/data/raw/Ecommerce.csv',
                           encoding='iso-8859-1',
                           low_memory=False)

#drop extra column
df_ecomm_raw = df_ecomm_raw.drop(columns = ['Unnamed: 8'], axis = 1)

# DATA DISCRIPTION

In [3]:
# Copy dataset
df0 = df_ecomm_raw.copy()

## Rename Columns

In [ ]:
cols_old = ['InvoiceNo','StockCode','Description','Quantity', 'InvoiceDate','UnitPrice','CustomerID','Country']

snakecase = lambda x: inflection.underscore(x)
col_news = list(map(snakecase, cols_old))

# Rename columns
df0.columns = col_news

## Data Dimension

In [ ]:
print('Number of rows: {}.'.format(df0.shape[0]))
print('Number of cols: {}.'.format(df0.shape[1]))

## Data Types

In [ ]:
df0.info()

## Check NA Values

In [ ]:
missing_count = df0.isnull().sum() # the count of missing values
value_count = df0.isnull().count() # the total values count

missing_percentage = round(missing_count/value_count*100,2) # the percentage of missing values
missing_df = pd.DataFrame({'missing value count': missing_count, 'percentage': missing_percentage})
missing_df

In [ ]:
barchart = missing_df.plot.bar(y='percentage')
for index, percentage in enumerate( missing_percentage ):
    barchart.text( index, percentage, str(percentage)+'%')

## Fillout NA

In [ ]:
# separate NA's in two different dataframe, one with NAs and other without it
df_missing = df0.loc[df0['customer_id'].isna(), :]
df_not_missing = df0.loc[~df0['customer_id'].isna(), :]

In [ ]:
# create reference
df_backup = pd.DataFrame( df_missing['invoice_no'].drop_duplicates().copy() )
df_backup['customer_id'] = np.arange( 19000, 19000+len( df_backup ), 1) # Fillout NA stratety: creating customers_id to keep their behavior (25% of the database)

# merge original with reference dataframe
df0 = pd.merge( df0, df_backup, on='invoice_no', how='left' )

# coalesce 
df0['customer_id'] = df0['customer_id_x'].combine_first( df0['customer_id_y'] )

# drop extra columns
df0 = df0.drop( columns=['customer_id_x', 'customer_id_y'], axis=1 )

In [ ]:
df0.isna().sum()

## Change Types

In [ ]:
# Transforme datatype of variable invoice_date to datetime
df0['invoice_date'] = pd.to_datetime(df0['invoice_date'])

In [ ]:
df0['customer_id'] = df0['customer_id'].astype('int64')

In [ ]:
df0.dtypes

## Descriptive Statistics

In [ ]:
df0.describe().T

In [ ]:
df0.describe(include = object).T

In [ ]:
num_attributes = df0.select_dtypes(include = np.number)
cat_attributes = df0.select_dtypes(exclude = [np.number, np.datetime64])

### Numerical Attributes

In [ ]:
# central tendency - mean, median
ct1 = pd.DataFrame(num_attributes.apply(np.mean)).T
ct2 = pd.DataFrame(num_attributes.apply(np.median)).T

# dispersion - desvio padrão, min, max, range, skew, kurtosis
d1 = pd.DataFrame(num_attributes.apply(np.std)).T
d2 = pd.DataFrame(num_attributes.apply(np.min)).T
d3 = pd.DataFrame(num_attributes.apply(np.max)).T
d4 = pd.DataFrame(num_attributes.apply(lambda x: x.max()-x.min())).T
d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T

#concatenate

m1 = pd.concat([d2,d3,d4,ct1,ct2,d1,d5,d6]).T.reset_index()
m1.columns = ['attributes', 'min', 'max', 'range', 'mean', 'mediana','std', 'skew','kurtosis']
m1

### Categorical Attributes

#### Invoice_No

In [ ]:
# problem: We got letters and numbers in invoice_no
#df1['invoice_no'].astype( int )

# identification: 
df_letter_invoices = df0.loc[df0['invoice_no'].apply( lambda x: bool( re.search( '[^0-9]+', x ) ) ), :]
df_letter_invoices.head()

print( 'Total number of invoices: {}'.format( len( df_letter_invoices ) ) )
print( 'Total number of negative quantity: {}'.format( len( df_letter_invoices[ df_letter_invoices['quantity'] < 0 ] ) ) )

#### Stock_Code

In [ ]:
# check stock codes only characters
df0.loc[df0['stock_code'].apply( lambda x: bool( re.search( '^[a-zA-Z]+$', x ) ) ), 'stock_code'].unique()

# Acão:
## 1. Remove stock_code in ['POST', 'D', 'M', 'PADS', 'DOT', 'CRUK']

# VARIABLE FILTERING

In [ ]:
df1 = df0.copy()

In [ ]:
 # === Numerical attributes ====
df1 = df1.loc[df1['unit_price'] >= 0.04, :]

# === Categorical attributes ====
df1 = df1[~df1['stock_code'].isin( ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY',
                                    'DCGSSGIRL', 'PADS', 'B', 'CRUK'] )]

# description
df1 = df1.drop( columns='description', axis=1 )

# country 
df1 = df1[~df1['country'].isin( ['European Community', 'Unspecified' ] ) ] #assuming this risk so we can use lat long parameters

# bad customers
df1 = df1[~df1['customer_id'].isin([16446])]

# quantity 
df1_returns = df1.loc[df1['quantity'] < 0, :].copy()
df1_purchases = df1.loc[df1['quantity'] >= 0, :].copy()

# FEATURE ENGINEERING

In [ ]:
df2 = df1.copy()

## Feature Creation

In [ ]:
# data reference
# RFM Model, creating feature for it

df_ref = df2.drop(['invoice_no', 'stock_code',
                   'quantity', 'invoice_date', 'unit_price',
                   'country'], axis = 1).drop_duplicates(ignore_index = True).copy()

### Gross Revenue

In [ ]:
# Gross Revenue ( Faturamento ) quantity * price
df1_purchases.loc[:, 'gross_revenue'] = df1_purchases.loc[:,'quantity'] * df1_purchases.loc[:, 'unit_price']

# Monetary (How much money a customer spends on purchases)
df_monetary = df1_purchases.loc[:, ['customer_id', 'gross_revenue']].groupby( 'customer_id' ).sum().reset_index()
df_ref = pd.merge( df_ref, df_monetary, on='customer_id', how='left' )
df_ref.isna().sum()

### Recency

In [ ]:
# Recency - Day from last purchase
df_recency = df1_purchases.loc[:, ['customer_id', 'invoice_date']].groupby( 'customer_id' ).max().reset_index()
df_recency['recency_days'] = ( df1['invoice_date'].max() - df_recency['invoice_date'] ).dt.days
df_recency = df_recency[['customer_id', 'recency_days']].copy()

df_ref = pd.merge( df_ref, df_recency, on='customer_id', how='left' )
df_ref.isna().sum()

### Qty Products  (different stock codes by customer)

In [ ]:
# Quantity of unique products purchased (Frequency: qntd of products over time)

# Number of products (different stock codes by customer)
df_freq = (df1_purchases.loc[:, ['customer_id', 'stock_code']].groupby( 'customer_id' ).count()
                                                           .reset_index()
                                                           .rename( columns={'stock_code': 'qty_products'} ) )
df_ref = pd.merge( df_ref, df_freq, on='customer_id', how='left' )
df_ref.isna().sum()

### Frequency

In [ ]:
#Frequency Purchase (rate: purchases by day)
df_aux = ( df1_purchases[['customer_id', 'invoice_no', 'invoice_date']].drop_duplicates()
                                                             .groupby( 'customer_id')
                                                             .agg( max_ = ( 'invoice_date', 'max' ), 
                                                                   min_ = ( 'invoice_date', 'min' ),
                                                                   days_= ( 'invoice_date', lambda x: ( ( x.max() - x.min() ).days ) + 1 ),
                                                                   buy_ = ( 'invoice_no', 'count' ) ) ).reset_index()
# Frequency
df_aux['frequency'] = df_aux[['buy_', 'days_']].apply( lambda x: x['buy_'] / x['days_'] if  x['days_'] != 0 else 0, axis=1 )

# Merge
df_ref = pd.merge( df_ref, df_aux[['customer_id', 'frequency']], on='customer_id', how='left' )
df_ref.isna().sum()

### Number of Returns

In [ ]:
#Number of Returns
df_returns = df1_returns[['customer_id', 'quantity']].groupby( 'customer_id' ).sum().reset_index().rename( columns={'quantity':'qty_returns'} )
df_returns['qty_returns'] = df_returns['qty_returns'] * -1

df_ref = pd.merge( df_ref, df_returns, how='left', on='customer_id' )
df_ref.loc[df_ref['qty_returns'].isna(), 'qty_returns'] = 0 #customers with 0 returned items

df_ref.isna().sum()

# EXPLORATORY DATA ANALYSIS (EDA)

In [ ]:
df3 = df_ref.dropna().copy()
df3.isna().sum()

## Space Study

In [ ]:
# Original dataset
#df33 = df3.drop(columns = ['customer_id'], axis = '').copy()

# dataset with selected columns due feature selection based on its importance
cols_selected = ['customer_id', 'gross_revenue', 'recency_days', 'qty_products', 'frequency', 'qty_returns']
df33 = df3[cols_selected].drop(columns = 'customer_id', axis = 1)

In [ ]:
df33.head()

In [ ]:
mm = pp.MinMaxScaler()

df33['gross_revenue'] = mm.fit_transform(df33[['gross_revenue']])
df33['recency_days']  = mm.fit_transform(df33[['recency_days']])
df33['qty_products']  = mm.fit_transform(df33[['qty_products']])
df33['frequency']     = mm.fit_transform(df33[['frequency']])
df33['qty_returns']   = mm.fit_transform(df33[['qty_returns']])

X = df33.copy()

In [ ]:
X.shape

#### PCA

In [ ]:
pca = dd.PCA( n_components = X.shape[1])


principal_components = pca.fit_transform(X)

# plot explained variables
features = range(pca.n_components_)

plt.bar(features, pca.explained_variance_ratio_, color = 'black') #quais componentes principais com a maior variação de dados

# pca component
df_pca = pd.DataFrame( principal_components )

In [ ]:
sns.scatterplot(x = 0, y = 1, data = df_pca);

#### UMAP

In [ ]:
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(X) #gera o espaço projetado - embedding é a projeção gerada em outro espaço

#embedding
df_umap = pd.DataFrame()
df_umap['embedding_X'] = embedding[:, 0]
df_umap['embedding_y'] = embedding[:, 1]

#plot UMAP - cluster projetado de alta dimencionalidade
sns.scatterplot(x = 'embedding_X', y = 'embedding_y',
                data = df_umap);

#### t-SNE

In [ ]:
reducer = mn.TSNE( n_components = 2, n_jobs = -1, random_state = 42)
embedding = reducer.fit_transform(X) #gera o espaço projetado - embedding é a projeção gerada em outro espaço

#embedding
df_tsne = pd.DataFrame()
df_tsne['embedding_X'] = embedding[:, 0]
df_tsne['embedding_y'] = embedding[:, 1]

#plot UMAP - cluster projetado de alta dimencionalidade
sns.scatterplot(x = 'embedding_X', y = 'embedding_y',
                data = df_tsne);

#### Tree-Based Embedding

In [ ]:
df3.head()

In [ ]:
# training dataset 
X = df33.drop(columns = ['gross_revenue'], axis = 1) #target variable
y = df33['gross_revenue']

# I could use boruta to select features to build a better embedding space

# model definition
rf_model = en.RandomForestRegressor(n_estimators = 100, random_state = 42)

# model training
rf_model.fit(X,y)

# leaf
df_leaf = pd.DataFrame(rf_model.apply( X ))

In [ ]:
# using UMAP to reduce the space study from 100 to 2
reducer = umap.UMAP(random_state = 42)
embedding = reducer.fit_transform(df_leaf) #gera o espaço projetado - embedding é a projeção gerada em outro espaço

#embedding
df_tree = pd.DataFrame()
df_tree['embedding_X'] = embedding[:, 0]
df_tree['embedding_y'] = embedding[:, 1]

#plot UMAP - cluster projetado de alta dimencionalidade
sns.scatterplot(x = 'embedding_X', y = 'embedding_y',
                data = df_tree);

# DATA PREPARATION

In [ ]:
# Tree-Based Embbeding
df4 = df_tree.copy()

# # UMAP Embbeding
# df4 = df_umap.copy()

# # TSNE Embedding
# df4 = df_tsne.copy()

# HYPERPARAMETER FINE-TUNNING

In [ ]:
X = df4.copy()

In [ ]:
X.head()

In [ ]:
clusters = np.arange(2, 31, 1) #silhouette was increasing, so we put more k points
clusters

## K-Means

In [ ]:
kmeans_sil = []

for k in clusters:
    # model definition
    kmeans_model = c.KMeans( n_clusters = k, n_init = 100, random_state = 42 )

    # model training
    kmeans_model.fit(X)

    # model predict
    labels = kmeans_model.predict(X)

    # model performance
    sil = m.silhouette_score( X, labels, metric = 'euclidean')
    kmeans_sil.append(sil)  

In [ ]:
plt.plot( clusters, kmeans_sil, linestyle = '--', marker = 'o', color = 'b' )
plt.xlabel( 'K' );
plt.ylabel('Silhouette Score');
plt.title('KMeans Silhouette Score per K ');

## GMM

In [ ]:
gmm_sil = []
for k in clusters:
    # model definition
    gmm_model = mx.GaussianMixture(n_components = k, n_init = 100, random_state = 42)
    
    # model training
    
    gmm_model.fit(X)
    
    # model prediction
    
    labels = gmm_model.predict(X)
    
    # model performance
    sil = m.silhouette_score(X, labels, metric = 'euclidean')
    gmm_sil.append(sil)

In [ ]:
plt.plot(clusters, gmm_sil, linestyle = '--', marker = 'o', color = 'b')
plt.xlabel( 'K' );
plt.ylabel('Silhouette Score');
plt.title('GMM Silhouette Score per K ');

## Hierarchical Clustering

In [ ]:
# model definition and training
hc_model = hc.linkage(X, 'ward')

### H-Clustering Silhouette Score

In [ ]:
hc_sil = []
for k in clusters:
    #model definition and training
    hc_model = hc.linkage(X, 'ward')

    # model predict
    labels = hc.fcluster(hc_model, k, criterion = 'maxclust')

    # metrics
    sil = m.silhouette_score(X, labels, metric = 'euclidean')
    hc_sil.append(sil)

In [ ]:
plt.plot(clusters, hc_sil, linestyle = '--', marker = 'o', color = 'b')

## Results

In [ ]:
## Results - Tree Based Embedding
df_results = pd.DataFrame({'KMeans:': kmeans_sil,
                           'GMM': gmm_sil,
                           'HC': hc_sil}
                           ).T

df_results.columns = clusters
df_results.style.highlight_max(color = 'lightgreen', axis = 1)

In [ ]:
## Results - UMAP Embedding
df_results = pd.DataFrame({'KMeans:': kmeans_sil,
                           'GMM': gmm_sil,
                           'HC': hc_sil}
                           ).T

df_results.columns = clusters
df_results.style.highlight_max(color = 'lightgreen', axis = 1)

In [ ]:
## Results - TSNE Embedding
df_results = pd.DataFrame({'KMeans:': kmeans_sil,
                           'GMM': gmm_sil,
                           'HC': hc_sil}
                           ).T

df_results.columns = clusters
df_results.style.highlight_max(color = 'lightgreen', axis = 1)

# MACHINE LEARNING MODEL TRAINING

## K-Means

In [ ]:
# model definition
k = 8;
kmeans = c.KMeans(init = 'random', n_clusters = k, n_init = 100, max_iter = 300, random_state = 42)

# model training
kmeans.fit(X)

# clustering
labels = kmeans.labels_

In [ ]:
# # trying with GMM beacuse of its approach in the embedding space
# # k=11 ;
# # model definition
# gmm_model = mx.GaussianMixture(n_components = k,n_init = 10 ,random_state=42)

# # model training
# gmm_model.fit(X)

# # model prediction
# labels = gmm_model.predict(X)

## Cluster Validation

In [ ]:
# WSS (Within-cluster Sum of Square )
# print('WSS score: {}'.format(kmeans.inertia_))

# SS (Silhouette Score)
print('SS score: {}'.format(m.silhouette_score(X, labels, metric = 'euclidean')))

# CLUSTER ANALYSIS

In [ ]:
df9 = X.copy()
df9['cluster'] = labels

## Visualization Inspection

In [ ]:
# k = 8 for  KMeans
sns.scatterplot(x = 'embedding_X', y = 'embedding_y', hue = 'cluster', data = df9, palette = 'deep')

## Cluster Profile

In [ ]:
df92 = df3[cols_selected].copy()
df92['cluster'] = labels
df92.head()

In [ ]:
# Explaining clusters profile based on this averages
# Number of customer
df_cluster = df92[['customer_id', 'cluster']].groupby( 'cluster' ).count().reset_index()
df_cluster['perc_customer'] = 100*( df_cluster['customer_id'] / df_cluster['customer_id'].sum() )

# Avg Gross revenue
df_avg_gross_revenue = df92[['gross_revenue', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_gross_revenue, how='inner', on='cluster' )

# Avg recency days
df_avg_recency_days = df92[['recency_days', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_avg_recency_days, how='inner', on='cluster' )

# Avg qty products
df_qty_products = df92[['qty_products', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qty_products, how='inner', on='cluster' )

# Frequency
df_frequency = df92[['frequency', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_frequency, how='inner', on='cluster' )

# Avg qty returns
df_qty_returns = df92[['qty_returns', 'cluster']].groupby( 'cluster' ).mean().reset_index()
df_cluster = pd.merge( df_cluster, df_qty_returns, how='inner', on='cluster' )

df_cluster

In [ ]:
# during the new EDA we can do a analyse inside each cluster

Cluster Insiders (04):

- Number of customers: 551 (9.67% of costumers)
- Avg Gross Revenue: $10410,00
- Recency Average: 45 days
- Avg of Qty Products Purchased: 366 un
- Purchase Frequency: 0.21 products per day

# Exploratory Dada Analysis

In [ ]:
df10 = df92.copy()

In [ ]:
df10.head()

# MODEL DEPLOYMENT

In [ ]:
df92.dtypes

In [ ]:
df92['recency_days'] = df92['recency_days'].astype(int)
df92['qty_products'] = df92['qty_products'].astype(int)
df92['qty_returns'] = df92['qty_returns'].astype(int)

In [ ]:
# # create database
# conn = sqlite3.connect('insiders_db.sqlite')

# # create table
# query_create_insiders = """
#     CREATE TABLE insiders (
#         custer_id     INTEGER,
#         gross_revenue REAL,
#         recency_days  INTEGER,
#         qty_products  INTEGER,
#         frequency     INTEGER,
#         qty_returns   INTEGER,
#         cluster       INTEGER
#         )
# """

# conn.execute(query_create_insiders)
# conn.commit()
# conn.close()

In [ ]:
# database connection
conn = create_engine('sqlite:///insiders_db.sqlite')

# # drop table
# query_drop_insiders = """
#     DROP TABLE insiders

# """

#create table
query_create_insiders = """
    CREATE TABLE insiders (
        customer_id   INTEGER,
        gross_revenue REAL,
        recency_days  INTEGER,
        qty_products  INTEGER,
        frequency     INTEGER,
        qty_returns   INTEGER,
        cluster       INTEGER
    )
"""

conn.execute(query_create_insiders)

In [ ]:
# insert into data
df92.to_sql('insiders', con = conn, if_exists = 'append', index = False)

In [ ]:
# consulting database
# get query
query_collect = """
    SELECT * from insiders
"""

df = pd.read_sql_query(query_collect, conn)
df.head()